เชื่อม google colab ก่อนทุกครั้ง

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


ทการดูว่าเเต่ละ category มีคำไหนที่มีความสำคัญบ้างโดยใช้เทคนิค TF - IDF

In [2]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# ✅ โหลดข้อมูล JSON ที่ถูกจัดหมวดแล้ว
with open('/content/drive/MyDrive/LockLearn/recommendations_with_categories.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# ✅ สร้าง DataFrame
df = pd.DataFrame(data)

# ✅ รวมคำแนะนำตาม category เป็นข้อความเดียว
grouped = df.groupby('category')['text'].apply(lambda x: ' '.join(x)).reset_index()

# ✅ ใช้ TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(grouped['text'])
feature_names = vectorizer.get_feature_names_out()

# ✅ สร้าง DataFrame ของ TF-IDF Matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=grouped['category'], columns=feature_names)

# ✅ ฟังก์ชันดู Top-k Keywords ของแต่ละ Category
def get_top_keywords(tfidf_df, category, k=10):
    row = tfidf_df.loc[category]
    return row.sort_values(ascending=False).head(k)

# ✅ แสดง Top Keywords ของแต่ละ Category
for cat in tfidf_df.index:
    print(f"🔎 Top keywords for '{cat}':")
    top_keywords = get_top_keywords(tfidf_df, cat)
    for word, score in top_keywords.items():
        print(f"{word:>15} : {score:.4f}")
    print("-" * 50)


🔎 Top keywords for 'Business':
        quality : 0.4749
      customers : 0.3801
       business : 0.3271
          value : 0.2979
     addressing : 0.1987
       priority : 0.1987
         points : 0.1900
     management : 0.1823
       services : 0.1823
        company : 0.1692
--------------------------------------------------
🔎 Top keywords for 'Business/Career Development':
        company : 0.3763
            add : 0.3520
        success : 0.3313
           long : 0.3313
           term : 0.3313
          value : 0.3313
          lives : 0.3313
        achieve : 0.3048
         people : 0.3048
        perfect : 0.0000
--------------------------------------------------
🔎 Top keywords for 'Career Development':
         career : 0.2325
          goals : 0.2308
       consider : 0.2180
            new : 0.2171
         skills : 0.1677
           work : 0.1638
        success : 0.1628
           help : 0.1581
        achieve : 0.1498
           make : 0.1433
--------------------------

In [ ]:
# รวม top-k keywords ของทุก category ลง DataFrame
def get_all_top_keywords(tfidf_df, k=10):
    rows = []
    for category in tfidf_df.index:
        top_keywords = tfidf_df.loc[category].sort_values(ascending=False).head(k)
        for word, score in top_keywords.items():
            rows.append({"Category": category, "Keyword": word, "TF-IDF": score})
    return pd.DataFrame(rows)

# แสดงเป็นตาราง
top_keywords_df = get_all_top_keywords(tfidf_df, k=10)
top_keywords_df.sort_values(by=["Category", "TF-IDF"], ascending=[True, False], inplace=True)

# แสดงผล
top_keywords_df.head(50)  # ปรับจำนวนได้
